In [2]:
# --------------------------------------------------------
# Tensorflow Faster R-CNN
# Licensed under The MIT License [see LICENSE for details]
# Written by Jiasen Lu, Jianwei Yang, based on code from Ross Girshick
# --------------------------------------------------------
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
from PIL import Image

import torchvision.transforms as transforms
import torchvision.datasets as dset
# from scipy.misc import imread
from imageio import imread
from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.rpn.bbox_transform import clip_boxes
# from model.nms.nms_wrapper import nms
from model.roi_layers import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections, vis_detections_PIL, vis_detections_filtered_objects_PIL, vis_detections_filtered_objects # (1) here add a function to viz
from model.utils.blob import im_list_to_blob
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet
import pdb

xrange = range  # Python 3


In [25]:
import pandas as pd
import os, glob

LABELS_DIR = os.path.expanduser('~/data/datasets/EPIC_KITCHENS_2018/custom_labels_30')
FRAMES_DIR = os.path.expanduser('~/data/datasets/EPIC_KITCHENS_2018/frames_30')
FEATS_DIR = os.path.expanduser('./feats')
FEATS_EXT = '_base.pt'

video_name = 'P06_02'
f_csv_labels = os.path.join(LABELS_DIR, video_name + '.csv')

df_labels = pd.read_csv(f_csv_labels)
df_labels

,original_vido_id,video_id,frame_id,path,labels
0,P06_02,3,0,train/P06/P06_02/P06_02_000001.jpg,0
1,P06_02,3,1,train/P06/P06_02/P06_02_000002.jpg,0
2,P06_02,3,2,train/P06/P06_02/P06_02_000003.jpg,0
3,P06_02,3,3,train/P06/P06_02/P06_02_000004.jpg,0
4,P06_02,3,4,train/P06/P06_02/P06_02_000005.jpg,0
...,...,...,...,...,...
416,P06_02,3,416,train/P06/P06_02/P06_02_000417.jpg,0
417,P06_02,3,417,train/P06/P06_02/P06_02_000418.jpg,0
418,P06_02,3,418,train/P06/P06_02/P06_02_000419.jpg,0
419,P06_02,3,419,train/P06/P06_02/P06_02_000420.jpg,0


In [31]:
all_frame_names = df_labels.path.apply(lambda x: os.path.splitext(os.path.basename(x))[0])
feat_files = all_frame_names.apply(lambda x: os.path.join(FEATS_DIR, x + FEATS_EXT))
feat_files_exist = feat_files.apply(os.path.exists)
df_feats = pd.DataFrame({'frame_name': all_frame_names, 'frame_path': df_labels.path, 'feat_files': feat_files,
                         'feats_exist': feat_files_exist})
df_feats

,frame_name,frame_path,feat_files,feats_exist
0,P06_02_000001,train/P06/P06_02/P06_02_000001.jpg,./feats/P06_02_000001_base.pt,True
1,P06_02_000002,train/P06/P06_02/P06_02_000002.jpg,./feats/P06_02_000002_base.pt,True
2,P06_02_000003,train/P06/P06_02/P06_02_000003.jpg,./feats/P06_02_000003_base.pt,True
3,P06_02_000004,train/P06/P06_02/P06_02_000004.jpg,./feats/P06_02_000004_base.pt,True
4,P06_02_000005,train/P06/P06_02/P06_02_000005.jpg,./feats/P06_02_000005_base.pt,True
...,...,...,...,...
416,P06_02_000417,train/P06/P06_02/P06_02_000417.jpg,./feats/P06_02_000417_base.pt,True
417,P06_02_000418,train/P06/P06_02/P06_02_000418.jpg,./feats/P06_02_000418_base.pt,True
418,P06_02_000419,train/P06/P06_02/P06_02_000419.jpg,./feats/P06_02_000419_base.pt,True
419,P06_02_000420,train/P06/P06_02/P06_02_000420.jpg,./feats/P06_02_000420_base.pt,True


In [33]:
print(df_feats.feats_exist.sum(), '/', len(df_feats.feats_exist), 'feature files exist.')

421 / 421 feature files exist


In [ ]:
frames_list = sorted(glob.glob(os.path.join(FRAMES_DIR, '*.jpg')))
assert not len(frames_list) == 0, f"No frames found in {FRAMES_DIR}"

feats_list = sorted(glob.glob(os.path.join(FEATS_DIR, '*.pt')))